In [2]:
import pandas as pd
import os

C:\Users\thuhi\AppData\Local\Temp\ipykernel_19080\3026406390.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
import sys
sys.path
os.getcwd()     

'c:\\Users\\thuhi\\workspace\\fraud_detection\\notebook'

In [2]:

import pandas as pd
pd.read_pickle(rf"C:\Users\thuhi\workspace\fraud_detection\data\raw_data\2018-09-28.pkl")

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
1725492,1725492,2018-09-28 00:00:26,3560,7244,76.95,15552026,180,0,0
1725493,1725493,2018-09-28 00:01:24,2790,2267,65.34,15552084,180,0,0
1725494,1725494,2018-09-28 00:01:27,4075,17,42.76,15552087,180,0,0
1725495,1725495,2018-09-28 00:01:58,4937,7230,7.81,15552118,180,0,0
1725496,1725496,2018-09-28 00:02:17,2266,6657,37.56,15552137,180,0,0
...,...,...,...,...,...,...,...,...,...
1735078,1735078,2018-09-28 23:56:17,3251,8118,36.53,15638177,180,0,0
1735079,1735079,2018-09-28 23:56:20,382,4840,16.65,15638180,180,0,0
1735080,1735080,2018-09-28 23:58:45,4675,8443,40.83,15638325,180,0,0
1735081,1735081,2018-09-28 23:59:07,637,4681,31.64,15638347,180,0,0


In [3]:

df = pd.DataFrame()
for file in os.listdir(rf"C:\Users\thuhi\workspace\fraud_detection\data\raw_data"):
    temp = pd.read_pickle(os.path.join(rf"C:\Users\thuhi\workspace\fraud_detection\data\raw_data", file))
    df = pd.concat([df, temp])

df.sort_values("TX_DATETIME",inplace=True)


In [ ]:

from src.components.features_engineering import Preprocessor
from src.exception import CustomException

preprocessor = Preprocessor(df)
preprocessor.get_CUSTOMER_ID_characteristic()



2025-05-17 22:43:06,473 - INFO - Initilizing get_CUSTOMER_ID_characteristic
2025-05-17 22:43:06,473 - INFO - Starting for 1d_WINDOW
2025-05-17 22:43:06,615 - INFO - Calculating WINDOW_TX_COUNT
2025-05-17 22:43:07,980 - INFO - Calculating WINDOW_TX_MEAN
2025-05-17 22:43:09,365 - INFO - Merging WINDOW_TX_COUNT into self.df
2025-05-17 22:43:09,474 - INFO - Merging WINDOW_TX_MEAN into self.df
2025-05-17 22:43:09,594 - INFO - Starting for 7d_WINDOW
2025-05-17 22:43:09,929 - INFO - Calculating WINDOW_TX_COUNT
2025-05-17 22:43:11,342 - INFO - Calculating WINDOW_TX_MEAN
2025-05-17 22:43:12,749 - INFO - Merging WINDOW_TX_COUNT into self.df
2025-05-17 22:43:12,880 - INFO - Merging WINDOW_TX_MEAN into self.df
2025-05-17 22:43:13,012 - INFO - Starting for 30d_WINDOW
2025-05-17 22:43:13,203 - INFO - Calculating WINDOW_TX_COUNT
2025-05-17 22:43:14,733 - INFO - Calculating WINDOW_TX_MEAN
2025-05-17 22:43:16,240 - INFO - Merging WINDOW_TX_COUNT into self.df
2025-05-17 22:43:16,364 - INFO - Merging WIN

In [5]:
preprocessor.df

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2018-04-02 01:00:01,596,0,57.16,31,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2018-04-02 09:49:55,4961,0,81.51,130,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018-04-03 12:14:41,2,0,146.00,476,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2018-04-05 16:47:41,4128,0,64.49,569,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2018-04-07 06:05:21,927,0,50.99,634,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1754150,1754150,2018-09-28 15:34:24,161,9999,54.24,15810996,182,0,0,3.0,0.0,6.0,0.0,26.0,0.0
1754151,1754151,2018-09-30 05:36:08,4342,9999,1.23,15811058,182,0,0,1.0,0.0,8.0,0.0,28.0,0.0
1754152,1754152,2018-09-30 09:00:10,618,9999,6.62,15811101,182,0,0,1.0,0.0,7.0,0.0,27.0,0.0
1754153,1754153,2018-09-30 17:47:37,4056,9999,55.40,15811192,182,0,0,1.0,0.0,7.0,0.0,27.0,0.0
